In [1]:
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_duffing import get_duffing_data
from sindy_utils import library_size
from training import train_network
import tensorflow as tf

# Generate data

In [2]:
# generate training, validation, testing data
noise_strength = 1e-6
training_data = get_duffing_data(1024, noise_strength=noise_strength)
validation_data = get_duffing_data(20, noise_strength=noise_strength)

# Set up model and training parameters

In [3]:
params = {}

params['input_dim'] = 128
params['latent_dim'] = 2
params['model_order'] = 1
params['poly_order'] = 3
params['include_sine'] = False
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0.0
params['loss_weight_sindy_x'] = 1e-4
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1024
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

# Run training experiments

In [4]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'lorenz_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING


2024-05-20 15:46:07.583194: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


Epoch 0
   training loss 272308.78125, (272307.9, 120192.53, 8614.258, 0.799702)
   validation loss 641707.375, (641706.3, 83981.086, 10469.697, 0.799702)
decoder loss ratio: 0.999323, decoder SINDy loss  ratio: 1.000342
Epoch 100
   training loss 242999.875, (242998.98, 31302.05, 8895.4795, 0.17626747)
   validation loss 607721.125, (607720.06, 121704.84, 10843.4795, 0.17626747)
decoder loss ratio: 0.946397, decoder SINDy loss  ratio: 1.036055
Epoch 200
   training loss 231119.734375, (231118.86, 6385.296, 8678.994, 0.15766224)
   validation loss 590734.4375, (590733.4, 6234.528, 10487.068, 0.15766224)
decoder loss ratio: 0.919944, decoder SINDy loss  ratio: 1.002001
Epoch 300
   training loss 272786.8125, (272785.94, 23728.791, 8621.935, 0.14583676)
   validation loss 642318.625, (642317.56, 11880.025, 10490.419, 0.14583676)
decoder loss ratio: 1.000275, decoder SINDy loss  ratio: 1.002322
Epoch 400
   training loss 232727.5, (232726.64, 4136.259, 8631.346, 0.124006465)
   validation

KeyboardInterrupt: 